# Exploring lineages and sublineages

## Setup

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from time import time

import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display

In [3]:
from speclet.io import DataFile, data_path, notebook_table_dir
from speclet.plot import set_speclet_theme

In [4]:
# Notebook execution timer.
notebook_tic = time()

# Plotting setup.
set_speclet_theme()
%config InlineBackend.figure_format = "retina"

# Constants
RANDOM_SEED = 847
np.random.seed(RANDOM_SEED)

In [5]:
output_dir = notebook_table_dir("001_020_lineage-exploration")

### Data

In [6]:
modeling_data_file = data_path(DataFile.DEPMAP_DATA)
modeling_data_file

PosixPath('/n/data1/hms/dbmi/park/Cook/speclet/modeling_data/depmap-modeling-data.csv')

In [7]:
modeling_data_dd = dd.read_csv(
    modeling_data_file, low_memory=False, dtype={"age": "float64"}
)
modeling_data_dd.head()

,sgrna,replicate_id,lfc,p_dna_batch,genome_alignment,hugo_symbol,screen,multiple_hits_on_gene,sgrna_target_chr,sgrna_target_pos,...,any_deleterious,any_tcga_hotspot,any_cosmic_hotspot,is_mutated,copy_number,lineage,lineage_subtype,primary_or_metastasis,is_male,age
0,AAACCTGCGGCGGTCGCCA,OVR3_c905R1,-0.299958,CRISPR_C6596666.sample,chr8_66505451_-,VXN,sanger,True,8,66505451,...,NaN,NaN,NaN,False,1.139595,ovary,ovary_adenocarcinoma,metastasis,False,60.0
1,AACAGCACACCGGCCCCGT,OVR3_c905R1,0.267092,CRISPR_C6596666.sample,chrX_156009834_-,IL9R,sanger,True,X,156009834,...,NaN,NaN,NaN,False,0.656377,ovary,ovary_adenocarcinoma,metastasis,False,60.0
2,AACCTCCGGACTCCTCAGC,OVR3_c905R1,0.550477,CRISPR_C6596666.sample,chr7_39609658_-,YAE1,sanger,True,7,39609658,...,NaN,NaN,NaN,False,0.923715,ovary,ovary_adenocarcinoma,metastasis,False,60.0
3,AACTCAAACTGACGCCGAA,OVR3_c905R1,-0.391922,CRISPR_C6596666.sample,chr1_117623388_-,TENT5C,sanger,True,1,117623388,...,NaN,NaN,NaN,False,1.352975,ovary,ovary_adenocarcinoma,metastasis,False,60.0
4,AACTGACCTTGAAACGCTG,OVR3_c905R1,-1.562577,CRISPR_C6596666.sample,chr16_66933623_+,CIAO2B,sanger,True,16,66933623,...,NaN,NaN,NaN,False,1.157211,ovary,ovary_adenocarcinoma,metastasis,False,60.0


## Analysis

In [8]:
lineage_data = (
    modeling_data_dd.query("screen == 'broad'")[
        ["depmap_id", "lineage", "lineage_subtype"]
    ]
    .drop_duplicates()
    .compute()
)
lineage_data

,depmap_id,lineage,lineage_subtype
86746,ACH-000004,blood,AML
157808,ACH-000005,blood,AML
228871,ACH-000007,colorectal,colorectal_adenocarcinoma
117813,ACH-000009,colorectal,colorectal_adenocarcinoma
25860,ACH-000011,urinary_tract,bladder_carcinoma
...,...,...,...
36493,ACH-002460,skin,melanoma
107555,ACH-002508,skin,melanoma
178617,ACH-002510,skin,melanoma
249679,ACH-002512,skin,melanoma


In [9]:
(
    lineage_data.copy()
    .assign(missing_subtype=lambda d: d["lineage_subtype"].isna())
    .groupby("lineage")["missing_subtype"]
    .sum()
    .reset_index()
    .set_index("lineage")
)

,missing_subtype
lineage,
bile_duct,0
blood,0
bone,0
breast,1
central_nervous_system,0
cervix,0
colorectal,0
epidermoid_carcinoma,0
esophagus,0


In [10]:
lineage_counts = (
    lineage_data.copy()
    .reset_index(drop=True)
    .fillna({"lineage_subtype": "NA"})
    .groupby(["lineage", "lineage_subtype"])["depmap_id"]
    .count()
    .reset_index()
    .sort_values(["lineage", "depmap_id"], ascending=(True, False))
)

for lineage, data in lineage_counts.groupby("lineage"):
    total = data["depmap_id"].sum()
    display(Markdown(f"**{lineage}** ({total} cell lines)"))
    display(data.reset_index(drop=True))
    display(Markdown("---"))

**bile_duct** (37 cell lines)

,lineage,lineage_subtype,depmap_id
0,bile_duct,cholangiocarcinoma,31
1,bile_duct,gallbladder_adenocarcinoma,6


---

**blood** (55 cell lines)

,lineage,lineage_subtype,depmap_id
0,blood,AML,26
1,blood,ALL,15
2,blood,CML,7
3,blood,CLL,4
4,blood,unspecified_leukemia,3


...

,lineage,lineage_subtype,depmap_id
0,urinary_tract,bladder_carcinoma,30


---

**uterus** (31 cell lines)

,lineage,lineage_subtype,depmap_id
0,uterus,endometrial_adenocarcinoma,18
1,uterus,choriocarcinoma,3
2,uterus,endometrial_squamous,3
3,uterus,MMMT,2
4,uterus,clear_cell_carcinoma,1
5,uterus,endometrial_adenosquamous,1
6,uterus,endometrial_stromal_sarcoma,1
7,uterus,mullerian_carcinoma,1
8,uterus,uterine_carcinosarcoma,1


---

In [11]:
lineage_counts.to_csv(output_dir / "broad-lineage-subtype-counts.csv", index=False)

---

## Session info

In [12]:
notebook_toc = time()
print(f"execution time: {(notebook_toc - notebook_tic) / 60:.2f} minutes")

execution time: 7.08 minutes


In [13]:
%load_ext watermark
%watermark -d -u -v -iv -b -h -m

Last updated: 2022-08-23

Python implementation: CPython
Python version       : 3.10.5
IPython version      : 8.4.0

Compiler    : GCC 10.3.0
OS          : Linux
Release     : 3.10.0-1160.45.1.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 28
Architecture: 64bit

Hostname: compute-e-16-231.o2.rc.hms.harvard.edu

Git branch: expand-lineages

dask      : 2022.7.1
matplotlib: 3.5.2
seaborn   : 0.11.2
pandas    : 1.4.3
numpy     : 1.23.1

